## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

Solve the given TSP instances using both a fast but approximate algorithm and a slower, yet more accurate one.

In [332]:
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random
from tqdm import tqdm

### Helper Functions

In [333]:
# Load data function that reads city coordinates and calculates the distance matrix
def load_data(country):
    CITIES = pd.read_csv(f'cities/{country}.csv', header=None, names=['name', 'lat', 'lon'])
    DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
    for c1, c2 in combinations(CITIES.itertuples(), 2):
        DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    return CITIES, DIST_MATRIX

In [334]:
# Function to calculate the total travel cost for a given tour (TSP solution)
def tsp_cost(tsp, CITIES, DIST_MATRIX):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

In [335]:
# Function to print the TSP route in a readable format with city names and distances
def print_tsp(tsp, CITIES, DIST_MATRIX):
    print("Miglior percorso trovato:")
    for c1, c2 in zip(tsp, tsp[1:]):
        print(f"{CITIES.at[c1,'name']} -> {CITIES.at[c2,'name']} ({DIST_MATRIX[c1,c2]:.2f} km)")

In [336]:
# Load city data and distance matrix for a specified country
country = 'vanuatu'
CITIES, DIST_MATRIX = load_data(country)

### Nearest Neighbor algorithm

In [337]:
def nearest_neighbor_tsp(CITIES, DIST_MATRIX):
    # Creates a copy of the distance matrix to avoid modifying the original
    dist = DIST_MATRIX.copy()
    # Initializes the starting city and an empty path list
    city = 0
    tsp_path = list()
    tsp_path.append(int(city))  # Adds the starting city to the TSP path

    # Loops until all cities are visited
    while not len(set(range(len(CITIES))) - set(tsp_path)) == 0:
        # Marks the current city's distances as infinity to prevent revisiting
        dist[:, city] = np.inf
        # Finds the closest city to the current one
        closest = np.argmin(dist[city])
        city = closest  # Updates the current city to the closest unvisited one
        tsp_path.append(int(city))  # Adds the closest city to the TSP path

    # Returns to the starting city to complete the cycle
    tsp_path.append(tsp_path[0])

    # Returns the completed path and calculates its total distance
    return tsp_path, tsp_cost(tsp_path, CITIES, DIST_MATRIX)


In [338]:
# Initializes the process by running the nearest-neighbor TSP and printing the result
tsp, cost = nearest_neighbor_tsp(CITIES, DIST_MATRIX)
print_tsp(tsp, CITIES, DIST_MATRIX)
print(f"\nTotal distance: {cost:.2f} km")

Miglior percorso trovato:
Isangel -> Vila (223.00 km)
Vila -> Lakatoro (206.74 km)
Lakatoro -> Norsup (2.46 km)
Norsup -> Luganville (67.09 km)
Luganville -> Port Olry (52.02 km)
Port Olry -> Longana (105.77 km)
Longana -> Sola (165.49 km)
Sola -> Isangel (652.96 km)

Total distance: 1475.53 km


### Genetic algorithm

In [372]:
# Definition of the Individual class
class Individual:
    def __init__(self, genome):
        self.genome = genome
        self.fitness = None 

# Fitness function based on path cost
def fitness(individual, CITIES, DIST_MATRIX):
    return -tsp_cost(individual.genome, CITIES, DIST_MATRIX)

# Parent selection function, choosing the best parent based on fitness
def parent_selection(population):
    candidates = sorted(np.random.choice(population, 4), key=lambda e: e.fitness, reverse=True)
    return candidates[0]

# Mutation that swaps two cities in the path
def single_mutation(p: Individual):
    genome = p.genome.copy()
    # Selects two random indices in the genome (excluding the first and last)
    i, j = random.sample(range(1, len(genome)-1), 2)
    # Swaps the cities at the chosen indices
    genome[i], genome[j] = genome[j], genome[i]
    return Individual(genome)

# Mutation that inverts a segment of the path
def inversion_mutation(p: Individual):
    genome = p.genome.copy()
    # Selects two random indices in the genome (excluding the first and last)
    i, j = np.random.choice(range(1, len(genome) - 1), size=2, replace=False)
    if i > j:  # Ensures i is less than j
        i, j = j, i
    # Reverses the segment of the genome between i and j
    genome[i:j] = genome[i:j][::-1]
    return Individual(genome)

# Order 1 Crossover (OX1) for the TSP
def OX1_xover(p1: Individual, p2: Individual):
    parent1 = p1.genome[1:-1].copy()  # Excludes the first and last cities (0)
    parent2 = p2.genome[1:-1].copy()

    size = len(parent1)
    # Randomly selects two points for the crossover
    start, end = sorted(np.random.choice(range(size), 2))
    # Creates an empty child array
    child = np.full(size, -1)
    # Copies the segment from the first parent
    child[start:end] = parent1[start:end]
    # Fills the remaining positions with cities from the second parent
    current_pos = end
    for city in parent2:
        if city not in child:
            if current_pos >= size:
                current_pos = 0  # Wraps around if the end is reached
            child[current_pos] = city
            current_pos += 1
    # Returns the child, adding 0 at the start and end for a complete path
    return Individual(np.concatenate(([0], child, [0])))

# Inver-Over Crossover function
def inver_over_xover(parent1: Individual, parent2: Individual) -> Individual:
    child_genome = parent1.genome.copy()
    # Selects a random city from the first parent (excluding start/end city 0)
    current_city = random.choice(parent1.genome[1:-1])
    # Finds the index of the current city in the child genome and in the second parent
    current_index = child_genome.index(current_city)
    next_city_index_parent2 = (parent2.genome.index(current_city) + 1) % len(parent2.genome)
    next_city_parent2 = parent2.genome[next_city_index_parent2]
    # Finds the index of the next city in the child genome
    next_city_index_child = child_genome.index(next_city_parent2)
    # Reverses the order of cities between the current and next city in the child genome
    child_genome[current_index + 1:next_city_index_child] = child_genome[current_index + 1:next_city_index_child][::-1]
    return Individual(child_genome)

In [370]:
def genetic_algorithm(country, CITIES, DIST_MATRIX):
    # Parameters for the genetic algorithm
    POPULATION_SIZE = 100
    OFFSPRING_SIZE = 50
    MAX_GENERATIONS = {
        'vanuatu': 1000,
        'italy': 10000,
        'russia': 10000,
        'us': 20000,
        'china': 30000,
    }
    crossover = inver_over_xover  # Crossover function to use
    mutation = inversion_mutation   # Mutation function to use

    # Generate an initial path using the nearest neighbor algorithm
    tsp = nearest_neighbor_tsp(CITIES, DIST_MATRIX)[0]

    # Create the initial population
    population = []
    for _ in range(POPULATION_SIZE):
        mutated_solution = tsp.copy()
        mutated_solution = inversion_mutation(Individual(mutated_solution)).genome
        population.append(Individual(mutated_solution))

    # Calculate the fitness for each individual in the initial population
    for ind in population:
        ind.fitness = fitness(ind, CITIES, DIST_MATRIX)

    # Main loop of the genetic algorithm, iterating for the defined number of generations
    for g in tqdm(range(MAX_GENERATIONS[country]), desc='Generations'):
        offspring = []
        # Adjust mutation rate that decreases over generations
        mutation_rate = max(0.1, (1 - g / MAX_GENERATIONS[country]) * 0.3)
        for _ in range(OFFSPRING_SIZE):
            if np.random.random() < mutation_rate:
                # If mutation is selected, choose a parent and apply mutation
                parent = parent_selection(population)
                offspring.append(mutation(parent))
            else:
                # If crossover is selected, choose two parents and apply crossover
                parent1 = parent_selection(population)
                parent2 = parent_selection(population)
                offspring.append(crossover(parent1, parent2))

        # Calculate fitness for the newly generated offspring
        for ind in offspring:
            ind.fitness = fitness(ind, CITIES, DIST_MATRIX)

        # Update the population by adding offspring and keeping the best individuals
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:POPULATION_SIZE]

    return population[0]  # Return the best individual found


In [341]:
# Execute the genetic algorithm and retrieve the best path
best_path = genetic_algorithm(country, CITIES, DIST_MATRIX)
print_tsp(best_path.genome, CITIES, DIST_MATRIX)
print(f"\nTotal distance: {-best_path.fitness:.2f} km")

Generations: 100%|██████████| 1000/1000 [00:08<00:00, 113.49it/s]

Miglior percorso trovato:
Isangel -> Vila (223.00 km)
Vila -> Lakatoro (206.74 km)
Lakatoro -> Norsup (2.46 km)
Norsup -> Luganville (67.09 km)
Luganville -> Port Olry (52.02 km)
Port Olry -> Longana (105.77 km)
Longana -> Sola (165.49 km)
Sola -> Isangel (652.96 km)

Total distance: 1475.53 km
